# Game Theory

Sander Aarts, 2020 (Based on *Lab 11: Matrix Games* by David Williamson, 2013)

**Objectives**
 - Introduce students to basic concepts in 0-sum 2-person games
 - Give students experience at computing optimal mixed minmax strategies
 - Give students experience at playing games

**Reading:** The data in this lab was found from the following Stats Zone articles: [Article 1](https://www.thestatszone.com/archive/statistical-insight-into-penalty-shootouts-where-to-place-your-penalty-13886) and [Article 2](https://www.thestatszone.com/archive/an-analyses-of-penalty-taking-trends-in-the-champions-league-13654)

**Brief description:** In this lab, you will consider penalty kicks in soccer as an example of a simple game. First, you
will be introduced to the rules and setting. Next, you will play several rounds of the game against the computer. Finally, you will formally solve this game by describing strategies available to the players and characterizing equilibrium strategies and payoffs.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

In [ ]:
# imports -- don't forget to run this cell
import pandas as pd
from ortools.linear_solver import pywraplp as OR

## Part 1: The Penalty Kick Game

In soccer (football) a penalty kick is awarded against a team if they commit a punishable foul inside their penalty area. The fouled team chooses one player that is allowed to take a free kick at the opposing team's goal while only the goalkeeper can defend. No other players may interfere, so the penalty kick can be seen as a one-shot game with two players. The kicker wants to kick the ball into the goal; the goalie wants to prevent this from happening. I.e. the two players' goals are diametrically opposed. Moreover, it typically takes the ball about 0.3 seconds to reach the goal from the time it is kicked, making it impossible for the goalkeeper to react to the kicker's action; both the kicker and the the goalkeeper must decide what to do without knowing the other player's action.

<div style="align: left; text-align:center;">
    <img src="images/carli_lloyd_goal.gif" alt="Carli Lloyd (US) scoring a penalty kick against Germany, 2015" title="Carli Lloyd (US) scoring a penalty kick against Germany, 2015." width="360px" />
    <div class="caption">Figure 1: A penalty kick. The kicker plays right; the goalkeeper left. It's a goal!
    </div>
</div>

You will play a simple version of the penalty kick game. The rules are as follows:
 - You are on offense. You will individually select different kickers (right or left footed) to shoot penalty kicks.
 - Your goal is to make as many goals as possible (maximize number of goals).
 - First, you select either a left- or right-footed kicker. The goalie is informed of which you picked.
 - Afterwards, the computer commits to an action; options are to dive **Left**, **Right**, or stay **Middle**
 - Before knowing the goalie's choice, you commit to kick to the goalie's **Left**, **Right**, or **Middle**.
 - The type of kicker, the goalie's action, and your action jointly determine the probability of a win.
 - A goal / no goal result is drawn according to this probability. A goal generates 1 point; a save -1. You want to maximize the total score.
 - The table below shows the scoring probabilities for all possible outcomes.

<div style="align: left; text-align:center;">
    <table style="width:60%">
  <tr>
    <th>Scoring Probability</th>
    <th>LL</th>
    <th>LM</th>
    <th>LR</th>
    <th>ML</th>
    <th>MM</th>
    <th>MR</th>
    <th>RL</th>
    <th>RM</th>
    <th>RR</th>
  </tr>
  <tr>
    <td>Right-footed kicker</td>
    <td>.55</td>
    <td>.65</td>
    <td>.93</td>
    <td>.74</td>
    <td>.60</td>
    <td>.72</td>
    <td>.95</td>
    <td>.73</td>
    <td>.70</td>
  </tr>
  <tr>
    <td>Left-footed kicker</td>
    <td>.67</td>
    <td>.70</td>
    <td>.96</td>
    <td>.74</td>
    <td>.60</td>
    <td>.72</td>
    <td>.87</td>
    <td>.65</td>
    <td>.61</td>
  </tr>
</table>
    <div class="caption">Table 1: The first letter in a column denotes the kicker's choice, the latter the goalie's choice. </div>
</div> 

<font color='red'> **TODO:** Make sure the penalty kick game is implemented correctly. More information written about this later on in the document.</font>

**Q:** Play 50 rounds against the computer [HERE](http://engri1101.orie.cornell.edu/penalty_kick_source/penalty_kick_game.html). The computer is not dumb
(or at least, it has not been programmed in a stupid way) and so it follows "fictitious play" by trying to play its best action against your past choices. When you finish the run of 50 times, the program will report your overall score.  Try to do as well as you can. (You must trust the software that the goals are determined honestly from the listed probabilities.) How well did you do? 

**A:** <font color='blue'> Answers will vary.</font>

## Part 2: Solving the Penalty Kick Game

In this part of the lab, we will try to analyze the game that you have just played. In formulating a 2-person
0-sum game, the first step is to figure out all of the pure strategies (not just the good ones) for each of
the players. Remember, one strategy can be thought of as a complete set of instructions that you can give
someone else, so that he or she plays just as you would until the end of the game (without knowing which
moves will be taken in advance).

**Q:** Give the six pure strategies that you (the striker) might have played.

**A:** <font color='blue'> You can choose the following pure strategies:
    
 - Pick a right-footed striker and kick Right (RR).
 - Pick a right-footed striker and kick Middle (RM).
 - Pick a right-footed striker and kick Left (RL).
 - Pick a left-footed striker and kick Right (LR).
 - Pick a left-footed striker and kick Middle (LM).
 - Pick a left-footed striker and kick Left (LL).
</font>

The pure strategies for the goalie are a little more complicated. After you pick a striker, their dominant foot is announced. At this point, the goalie only has three pure strategies available; Right, Middle, and Left. Imagine we give the goalie explicit instructions which denote exactly what to do for any possibile action the striker may take.

**Q:** For what actions from the striker do we need to specify a response for the goalie?

**A:** <font color='blue'>We must tell the goalie what to do in the case of a left-footed kicker and what to do in the case of a right-footed kicker.</font>

Following the notation from the handout, a pure strategy from the goalie takes the form $([L, i], [R, j])$ read as "If the kicker is left-footed, do $i$. If the kicker is right-footed, do $j$."

**Q:** What values can $i$ and $j$ take? How many pure stratagies does the goalie have?

**A:** <font color='blue'> Both $i$ and $j$ can take values in ${L, M, R}$. Hence, the goalie has 9 pure strategies.</font>

Now, we break the game down into sub-games to make things easier. We have two games: one for the left-footed kicker and one for the right-footed kicker.

**Q:** Write down the 3x3 payoff matrices for **both** games.

**A:** <font color='blue'> Answer revealed below. </font>

**Q:** In each matrix, are any pure strategies dominated by another pure strategy?

**A:** <font color='blue'>No.</font>

Run the following cell to see the correct payoff matrices for both the left-footed kicker and right-footed kicker sub-games. Here, the striker is the row player and the goalie is the column player. If they do not match your answer, consider why. Use these payoff matrices for the following computations.

In [ ]:
right = pd.read_csv('data/right_subgame.csv', index_col=0)
left = pd.read_csv('data/left_subgame.csv', index_col=0)
display(right)
display(left)

In lecture and the handout, you learned about dominance by a pure strategy. However, we can also have dominance by a mixed strategy.

**Q:** Consider the right-footed striker sub-game. What is the expected payoff if the striker does a 50/50 mixed strategy between kicking left ($L$) and kicking right ($R$)? You should give the expected probability of a goal in the case the goalie goes right ($R$), middle ($M$), or left ($L$).

**A:**  <font color='blue'>The expected payoff is $\frac{1}{2}(0.55) + \frac{1}{2}(0.95) = 0.75$ for $L$, $\frac{1}{2}(0.65) + \frac{1}{2}(0.73) = 0.69$ for $M$, and $\frac{1}{2}(0.93) + \frac{1}{2}(0.70) = 0.815$ for $R$ </font>

**Q:** Does the striker's 50/50 mixed strategy between $L$ and $R$ dominate any pure strategy? If so, which one?

**A:**  <font color='blue'>It dominates the pure strategy in which the striker kicks middle ($M$).</font>

You should find that the striker's pure strategy of kicking middle is dominated. Hence, we can remove it from the payoff matrix as it will never be played. This leaves us with the simplified payoff matrix below.

In [ ]:
right.drop('M')

**Q:** Consider the goalie in the right-footed striker sub-game. Does a 50/50 mix between left $(L)$ and right ($R$) dominate the pure strategy of middle ($M$)? Why or why not?

**A:** <font color='blue'> No, the goalie wants to minimize the probability of a goal unlike the striker. </font>   

**Q:** Now consider the left-footed striker sub-game. Eliminate any dominated pure strategies. You should be able to simplify the payoff matrix to a 2x3. (Hint: this payoff matrix simplifies in a similar fashion to the right-footed striker payoff matrix.)

**A:** <font color='blue'> Again, the striker's pure strategy of $M$ is dominated by a 50/50 mix between the other two pure strategies. The simplified payoff matrix is: </font> 

In [ ]:
### BEGIN SOLUTION
left.drop('M')
### END SOLUTION

In [ ]:
# Right-foot striker payoff matrix for reference
right.drop('M')

**Q:** Let's return to the right-footed striker subgame (with the simplified payoff matrix given above). Assume the striker kicks left ($L$) with probability $p_L$ and kicks right ($R$) with probability $p_R$. What is the expected payoff associated with each of the goalie's pure strategies ($L$, $M$, $R$)?

**A:** <font color='blue'> ($L$) Payoff is $p_L(0.55) + p_R(0.95)$. 
($M$) Payoff is $p_L(0.65) + p_R(0.73)$.
($R$) Payoff is $p_L(0.93) + p_R(0.70)$. </font>  

**Q:** Simplify your expressions above so that they just depend on the variable $p_L$. (Hint: What is the sum of $p_L$ and $p_R$?)

<font color='blue'> $p_L(0.55) + p_R(0.95) = p_L(0.55) + (1 - p_L)(0.95) = (0.55)p_L + 0.95 - (0.95)p_L = 0.95 - (0.4)p_L$
    $p_L(0.65) + p_R(0.73) = p_L(0.65) + (1 - p_L)(0.73) =  (0.65)p_L + 0.73 - (0.73)p_L = 0.73 - (0.08)p_L$
   $p_L(0.93) + p_R(0.70) = p_L(0.93) + (1 - p_L)(0.70) = (0.93)p_L + 0.70 - (0.70)p_L = 0.70 + (0.23)p_L$  </font>

**Q:** As the striker, we want to find the mixed strategy which maximizes the minimum probability we score. Using the graphical method, what values of $p_L$ and $p_R$ acheive this? What is the minimum expected probability we score? ([Desmos](https://www.desmos.com/calculator) is a helpful tool!)

**A:** <font color='blue'> $p_L = 0.0968, p_R = 0.9032$. The minimum expected probability we score is $0.7223$ </font>

**Q:** In **Q13**, you found the mixed strategy for the striker that maximized the minimum probability we scored. In solving this graphically, you should have graphed three lines. What did each of these lines correspond to?

**A:** <font color='blue'> Each line corresponds to a different pure strategy from the goalie. </font>

In addition to solving graphically, we can find the mixed strategy which maximizes the minimum probability we score using linear programming. We wish to maximize the minimum expected payoff. The expected payoff depends on which strategy the goalie uses. Hence, we want to maximize the minimum of $p_L(0.55) + p_R(0.95)$ (goalie uses $L$), $p_L(0.65) + p_R(0.73)$ (goalie uses $M$), and $p_L(0.93) + p_R(0.70)$ (goalie uses $R$) where $p_R + p_L = 1$ and $p_L,p_R \geq 0$. Recall, we can introduce another variable $z$ and enforce that the expected payoff for each of the goalie's pure strategies is $\geq z$. We then maximize $z$. Understand why this has the desired effect before proceeding. 

The linear program we just described is given below. Run the cells below to solve it. Verify that the linear programming solution matches the one you found via the graphical approach.

<font color='red'> <b>The cell above asks the student to understand how the variable z works for making the linear program before proceeding, but a cell below explains the idea behind it. If the student is expected to know this before proceeding, then there is no need for the explanation below, but if they are not expected to know this, then the explanation should be moved up to this cell while the cell that contains the explanation below should tell the student to understand why using z has the desired effect before proceeding.</b></font>

In [ ]:
def right_subgame_kicker():
    """A linear program for solving the right penalty kick subgame for the kicker."""
    # define the model
    m = OR.Solver('right_subgame-kicker', OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables
    R = m.NumVar(0, m.infinity(), 'R');
    L = m.NumVar(0, m.infinity(), 'L');
    z = m.NumVar(-m.infinity(), m.infinity(), 'z');

    # objective function.
    m.Maximize(z);

    # constraints.
    m.Add(0.55*L + 0.95*R >= z);
    m.Add(0.65*L + 0.73*R >= z);
    m.Add(0.93*L + 0.70*R >= z);
    m.Add(L + R == 1);
    
    return m, m.variables()

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
m,x = right_subgame_kicker()
solve(m)

Let's consider the dual values of the linear program we just solved. Recall the economic interpretation of dual values which you were taught in lecture. The dual value of some constraint can be interpretted as the *marginal cost* of changing the requirement (i.e. the RHS) by 1. This interpretation has an interesting corollary. Consider an optimal solution in which some constraint is satisfied with strict inequality ($<$ or $>$). Slight changes to constraint (the RHS) will have no affect on the optimal solution, and thus, the optimal objective value. 

**Q:** Take this one step further. If slight changes to a constraint have no affect on the optimal objective value, what does that imply about the *marginal cost* of changing this requirement? What does that imply about the dual value of this constraint?

**A:** <font color='blue'> The *marginal cost* of changing this requirement is 0 and the dual value of the constraint is 0.</font>

In **Q**, you should have noticed that each constraint corresponds to a pure strategy from the goalie. Furthermore, we have seen that the LP used to find the optimal mixed strategy for the goalie (the one that minimizes the maximum probability of a goal) will just be the dual of the LP used to find the optimal mixed strategy of the stiker. Hence, the optimal dual values of the constraints correspond to an optimal mixed stategy for the goalie.  

**Q:** Consider the optimal solution to the LP used to find the optimal mixed strategy of the striker. Which constraint is held with strict inequality for that solution? Which pure strategy from the goalie does this constraint correspond to? What does the result from **Q15** imply about how often the goalie plays this pure strategy in the corresponding dual optimal solution 

**A:** <font color='blue'> The constraint $0.55*L + 0.95*R >= z$ holds with strict inequality. This corresponds to the goalie pure strategy of $L$. **Q15** implies that the goalie never plays this strategy in the corresponding dual optimal solution. </font>

We now know there exists an optimal goalie mixed strategy in which the goalie never uses the pure strategy of $L$. Hence, we simplify our payoff matrix by removing the column associated with this pure strategy. This allows us to solve from the goalie's perspective using the graphical method.

In [ ]:
right.drop('M').iloc[:,1:]

**Q:** The goalie wants to find the mixed strategy which minimizes the maximum probability we (the striker) score. Assume the striker kicks middle ($M$) with probability $q_M$ and kicks right ($R$) with probability $q_R$. Using the graphical method, what values of $q_M$ and $q_R$ acheive this? What is the maximum expected probability we are scored on? ([Desmos](https://www.desmos.com/calculator) is a helpful tool!)

**A:** <font color='blue'>($L$) Payoff is $q_M(0.65) + q_R(0.93)$. ($R$) Payoff is $q_M(0.73) + q_R(0.70)$
    
   $q_M(0.65) + q_R(0.93) = (0.65)q_M + (1 - q_M)(0.93) = 0.93 - q_M(0.28)$
    
   $q_M(0.73) + q_R(0.70) = (0.73)q_M + (1 - q_M)(0.70) = 0.70 + q_M(0.03)$

Hence, $q_M = 0.7419, q_R = 0.2581$. The maximum expected probability we score is $0.7223$.</font>

<font color='red'> <b>While the above solution is correct, I think there may be an issue with the problem. The striker should only be kicking to the left or right, and the goalie should be going only to the middle or right. Should these question be something more like the next cell?:</b></font>

**Q:** The goalie wants to find the mixed strategy which minimizes the maximum probability we (the striker) score. Assume the goalie goes middle ($M$) ith probability $q_M$ and goes right ($R$) with probability $q_R$. Using the graphical method, what values of $q_M$ and $q_R$ achieve this? What is the maximum expected probability for us to score on the goalie? ([Desmos](https://www.desmos.com/calculator) is a helpful tool!)

**A:** <font color='blue'>($L$) Payoff is $q_M(0.65) +q_R(0.93)$. ($R$) Payoff is $q_M(0.73) + q_R(0.70)$
    
   $q_M(0.65) + q_R(0.93) = (0.65)q_M + (1 - q_M)(0.93) = 0.93 - q_M(0.28)$
    
   $q_M(0.73) + q_R(0.70) = (0.73)q_M + (1 - q_M)(0.70) = 0.70 + q_M(0.03)$

Hence, $q_M = 0.7419, q_R = 0.2581$. The maximum expected probability we score is $0.7223$.</font>

**Q:** In **Q13**, you maximized the minimum expected probability of a goal. In **Q17**, you minimized the maximum expected probability of a goal. How did these two values relate. Did you expect this?

**A:** <font color='blue'> They are equal. This is expected by the minimax theorem.</font>

For the left-footed striker sub-game, let's use linear programming rather than the graphical method to find a mixed-strategy equilibrium! As a breif reminder, there is a special trick that is helpful in formulating LPs to solve this problem. Let's say we have 2 variables: $x$, $y$ and some constraints on them. We want to maximize the smaller of these two variables. Notationally, we can say $\max_{x,y} \min\{x,y\}$. For example, if $(x,y)= (2,3)$ would have an objective value of 2. The LP below achieves exactly this! (where "..." are the rest of the constraints)

$$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & x \geq z \\
\quad & y \geq z \\
\quad  ...
\end{align*}$$

In [ ]:
# Left-foot striker payoff matrix for reference
left.drop('M')

**Q:** Assume the striker kicks left ($L$) with probability $p_L$ and kicks right ($R$) with probability $p_R$. What is the expected payoff associated with each of the goalie's pure strategies ($L$, $M$, $R$)? (Hint: Remember we are thinking about the left-footed kicker now!)

**A:** <font color='blue'>($L$) Payoff is $p_L(0.67) + p_R(0.87)$. ($M$) Payoff is $p_L(0.70) + p_R(0.65)$. ($R$) Payoff is $p_L(0.96) + p_R(0.61)$.  </font>

**Q:** As the striker, we want to find the mixed strategy which maximizes the minimum probability we score. Write a linear program to solve this problem.

**A:** <font color='blue'> $$\begin{align*}
\max \quad & z\\
\text{s.t.} \quad & p_L(0.67) + p_R(0.87) \geq z \\
\quad & p_L(0.70) + p_R(0.65) \geq z \\
\quad & p_L(0.96) + p_R(0.61) \geq z \\
\quad & p_L + p_R = 1 \\
\quad & p_L, p_R \geq 0 \\
\end{align*}$$ </font>

**Q:** Complete the model below to implement your linear program from **Q19** as an OR-Tools model and use the cell below to solve it.

In [ ]:
def left_subgame_kicker():
    """A linear program for solving the left penalty kick subgame for the kicker."""
    # define the model
    m = OR.Solver('left_subgame-kicker', OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables
    R = m.NumVar(0, m.infinity(), 'R');
    L = m.NumVar(0, m.infinity(), 'L');
    z = m.NumVar(-m.infinity(), m.infinity(), 'z');

    # objective function
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()
    
    ### BEGIN SOLUTION
    m.Maximize(z);
    ### END SOLUTION

    # constraints
    # TODO: Define the constraints.
    # m.Add()
    
    ### BEGIN SOLUTION
    m.Add(0.67*L + 0.87*R >= z);
    m.Add(0.70*L + 0.65*R >= z);
    m.Add(0.96*L + 0.61*R >= z);
    m.Add(L + R == 1);
    ### END SOLUTION
    
    return m, m.variables()

In [ ]:
m,x = left_subgame_kicker()
solve(m)

**Q:** What is an optimal mixed-strategy for the (left-footed) striker? What is the minimum expected probability of a goal for this strategy?

**A:** <font color='blue'> $p_R = 0.12$ and $p_L = 0.88$ The minimum expected probability of a goal for this strategy is $0.694$. </font>

**Q:** The goalie wants to find the mixed strategy which minimizes the maximum probability we (the striker) score. Assume the goalie goes left ($L$) with probability $q_L$, goes middle ($M$) with probability $q_M$, and goes right ($R$) with probability $q_R$. Finish the model below to solve this problem. (Hint: it is just the dual of your previous one!)  

In [ ]:
def left_subgame_goalie(): 
    """A linear program for solving the left penalty kick subgame for the goalie."""
    # define the model
    m = OR.Solver('left_subgame-goalie', OR.Solver.CLP_LINEAR_PROGRAMMING);

    # decision variables.
    R = m.NumVar(0, m.infinity(), 'R');
    M = m.NumVar(0, m.infinity(), 'M');
    L = m.NumVar(0, m.infinity(), 'L');
    z = m.NumVar(-m.infinity(), m.infinity(), 'z');

    # objective function
    # TODO: Define the objective function.
    # m.Minimize() or m.Maximize()

    ### BEGIN SOLUTION
    m.Minimize(z);
    ### END SOLUTION

    # constraints
    # TODO: Define the constraints.
    # m.Add()

    ### BEGIN SOLUTION
    m.Add(0.67*L + 0.70*M + 0.96*R <= z);
    m.Add(0.87*L + 0.65*M + 0.61*R <= z);
    m.Add(L + M + R == 1);
    ### END SOLUTION
    
    return m, m.variables()

In [ ]:
m,x = left_subgame_goalie()
solve(m)

**Q:** What is an optimal mixed-strategy for the goalie (with left-footed kicker)? What is the minimum expected probability of a goal for this strategy?

**A:** <font color='blue'> $q_R = 0$, $q_M = 0.8$, and $q_L = 0.2$ The minimum expected probability of a goal for this strategy is $0.694$. </font>

**Q:** Assume the striker always uses the optimal mixed-strategy. Is one type of striker more likely to score a goal than the other striker?

**A:**  <font color='blue'> Yes, the right-footed striker scores with probability $0.7223 > 0.694$. </font>

**Q:** Collect your findings from above to compile a <b>complete set of instructions</b> for the players in equilibrium. (Hint: even if one type of kicker is always chosen, you should specify what the goalkeeper should do against the other type).

**A:** <font color='blue'> The instructions are:
    
 - Striker is always right-footed and with probabilities: $p_L = 0.0968$ and $p_R = 0.9032$
 - If right-footed striker, the goalkeeper with probabilities: $q_M = 0.7419$ and $q_R = 0.2581$
 - If left-footed striker, the goalkeeper with probabilities: $q_M = 0.8$ and $q_L = 0.2$
</font>

**Q:** Does the answer to **Q25** make intuitive sense?

**A:** <font color='blue'> Yes, the striker shoots in the dominant foot direction. Unlike real penalty kicks, the goalie often does not dive either direction.  </font>

**Q:** Play 50 rounds of the [GAME](http://engri1101.orie.cornell.edu/penalty_kick_source/penalty_kick_game.html) again using the optimal strategy you found. Pay attention to the goalie's choices. Does there strategy reflect what you would expect? Did you do better or worse than the first time you played? Was the score what you expected?

**A:** <font color='blue'> I would expect we get a goal ~72% of the time so 36 goals and 14 saves resulting in a score of 22. I got lucky and got 26 when I played. The goalie stayed middle the entire time though... </font>

<font color='red'> <b>The game is mentioned to use fictitious play at the beginning of the handout. Because fictitious play is different to a mixed strategy, the student cannot expect to get the same results as they would against a mixed strategy goalie. The goalie using fictitious play is what causes the behaviour of staying in the middle, because it views the middle as being the least dangerous position to take (least likelihood of having the player score) after playing against the player long enough (It will not take middle if it detects a better strategy IE: if the player always only goes right on a right footed striker, it will also only go right for right footed strikers.) As the goalie calculates its perceived chances of being scored on for each position in order to make its decision, the goalie can be said to use a different pure or mixed strategy each round (It will use a random chance if the perceived danger for two positions is equal).</b></font>

## Part 3: Full Model

In **Part 2**, we divided the problem into subgames first and then used dominance to simplify the two payoff matrices. However, we could fomulate the following linear program to solve the entire model.

In [ ]:
def penalty_kick_game(integer = False):
    """A linear program for solving a penalty kick game."""
    # define the model
    m = OR.Solver('penalty_kick_game', OR.Solver.CLP_LINEAR_PROGRAMMING);

    RR = m.NumVar(0, m.infinity(), 'RR'); #Chance of right footed striker shooting right
    RM = m.NumVar(0, m.infinity(), 'RM'); #Chance of right footed striker shooting middle
    RL = m.NumVar(0, m.infinity(), 'RL'); #Chance of right footed striker shooting left
    LR = m.NumVar(0, m.infinity(), 'LR'); #Chance of left footed striker shooting right
    LM = m.NumVar(0, m.infinity(), 'LM'); #Chance of left footed striker shooting middle
    LL = m.NumVar(0, m.infinity(), 'LL'); #Chance of left footed striker shooting left
    
    z = m.NumVar(-m.infinity(), m.infinity(), 'z'); #total probability of scoring

    # objective function
    m.Maximize(z);

    # constraints
    
    #Chance of scoring if goalie follows ([R,R],[L,R]) strategy is <= 
    #Chance of shooting right if right footed * 0.70 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if right footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if right footed * 0.93 (Chance of scoring if shot left and goalie goes right)
    # + Chance of shooting right if left footed * 0.61 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if left footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if left footed * 0.96 (Chance of scoring if shot left and goalie goes right)
    m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.61*LR + 0.72*LM + 0.96*LL >= z, name='([R,R],[L,R])');
    
    #Chance of scoring if goalie follows ([R,R],[L,M]) strategy is <= 
    #Chance of shooting right if right footed * 0.70 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if right footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if right footed * 0.93 (Chance of scoring if shot left and goalie goes right)
    # + Chance of shooting right if left footed * 0.65 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if left footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if left footed * 0.70 (Chance of scoring if shot left and goalie goes middle)
    m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.65*LR + 0.60*LM + 0.70*LL >= z, name='([R,R],[L,M])');
    
    #Chance of scoring if goalie follows ([R,R],[L,L]) strategy is <= 
    #Chance of shooting right if right footed * 0.70 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if right footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if right footed * 0.93 (Chance of scoring if shot left and goalie goes right)
    # + Chance of shooting right if left footed * 0.87 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if left footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if left footed * 0.67 (Chance of scoring if shot left and goalie goes left)
    m.Add(0.70*RR + 0.72*RM + 0.93*RL + 0.87*LR + 0.74*LM + 0.67*LL >= z, name='([R,R],[L,L])');
    
    #Chance of scoring if goalie follows ([R,M],[L,R]) strategy is <= 
    #Chance of shooting right if right footed * 0.73 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if right footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if right footed * 0.65 (Chance of scoring if shot left and goalie goes middle)
    # + Chance of shooting right if left footed * 0.61 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if left footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if left footed * 0.96 (Chance of scoring if shot left and goalie goes right)
    m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.61*LR + 0.72*LM + 0.96*LL >= z, name='([R,M],[L,R])');
    
    #Chance of scoring if goalie follows ([R,M],[L,M]) strategy is <= 
    #Chance of shooting right if right footed * 0.73 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if right footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if right footed * 0.65 (Chance of scoring if shot left and goalie goes middle)
    # + Chance of shooting right if left footed * 0.65 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if left footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if left footed * 0.70 (Chance of scoring if shot left and goalie goes middle)
    m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.65*LR + 0.60*LM + 0.70*LL >= z, name='([R,M],[L,M])');
    
    #Chance of scoring if goalie follows ([R,M],[L,L]) strategy is <= 
    #Chance of shooting right if right footed * 0.73 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if right footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if right footed * 0.65 (Chance of scoring if shot left and goalie goes middle)
    # + Chance of shooting right if left footed * 0.87 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if left footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if left footed * 0.67 (Chance of scoring if shot left and goalie goes left)
    m.Add(0.73*RR + 0.60*RM + 0.65*RL + 0.87*LR + 0.74*LM + 0.67*LL >= z, name='([R,M],[L,L])');
    
    #Chance of scoring if goalie follows ([R,L],[L,R]) strategy is <= 
    #Chance of shooting right if right footed * 0.95 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if right footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if right footed * 0.55 (Chance of scoring if shot left and goalie goes left)
    # + Chance of shooting right if left footed * 0.61 (Chance of scoring if shot right and goalie goes right)
    # + Chance of shooting middle if left footed * 0.72 (Chance of scoring if shot middle and goalie goes right)
    # + Chance of shooting left if left footed * 0.96 (Chance of scoring if shot left and goalie goes right)
    m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.61*LR + 0.72*LM + 0.96*LL >= z, name='([R,L],[L,R])');
    
    #Chance of scoring if goalie follows ([R,L],[L,M]) strategy is <= 
    #Chance of shooting right if right footed * 0.95 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if right footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if right footed * 0.55 (Chance of scoring if shot left and goalie goes left)
    # + Chance of shooting right if left footed * 0.65 (Chance of scoring if shot right and goalie goes middle)
    # + Chance of shooting middle if left footed * 0.60 (Chance of scoring if shot middle and goalie goes middle)
    # + Chance of shooting left if left footed * 0.70 (Chance of scoring if shot left and goalie goes middle)
    m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.65*LR + 0.60*LM + 0.70*LL >= z, name='([R,L],[L,M])');
    
    #Chance of scoring if goalie follows ([R,L],[L,L]) strategy is <= 
    #Chance of shooting right if right footed * 0.95 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if right footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if right footed * 0.55 (Chance of scoring if shot left and goalie goes left)
    # + Chance of shooting right if left footed * 0.87 (Chance of scoring if shot right and goalie goes left)
    # + Chance of shooting middle if left footed * 0.74 (Chance of scoring if shot middle and goalie goes left)
    # + Chance of shooting left if left footed * 0.67 (Chance of scoring if shot left and goalie goes left)
    m.Add(0.95*RR + 0.74*RM + 0.55*RL + 0.87*LR + 0.74*LM + 0.67*LL >= z, name='([R,L],[L,L])');


    m.Add(RR + RM + RL + LR + LM + LL == 1);
    
    return m, m.variables()

In [ ]:
m,x = penalty_kick_game()
solve(m)

Furthermore, rather than defining another model to get the goalie's optimal mixed strategy, we can get the probabilities from the dual values of this LP.

(**Note**: We look at the negative of the dual values since we flip the $\geq$ constraints to $\leq$)

In [ ]:
for c in m.constraints():
    print(c.name(),':', -c.dual_value())

**Q:** Compile a <b>complete set of instructions</b> for the players in equilibrium based on the optimal solution to the full LP model and the dual values.

**A:** <font color='blue'> The instructions are:
    
 - Striker is always right-footed and with probabilities: $p_L = 0.0968$ and $p_R = 0.9032$
 - If right-footed striker, the goalkeeper with probabilities: $q_M = 0.7419$ and $q_R = 0.2581$
</font>

<font color='red'> <b>The above solution ignores the left footed striker instructions for the goalie. Should the reason for their exclusion be included? (The left side instructions cannot use the dual in this case due to the fact that the striker will never be left-footed, so no matter what the left sided actions of the goalie are, the objective value to the solution will stay the same. Any combination of [R,M] constraints with dual values adding up to 0.7419 should still give an optimal solution).</b></font>

**Q:** Do these instructions match those you described in **Q25**? 

**A:** <font color='blue'> Yes. </font>